# D210 Data Dashboard and Storytelling Assessment — Task 1
### NAM Task 1: Data Dashboard And Storytelling
#### Representation and Reporting — D210
#### PRFA — NAM2
> André Davis
> StudentID: 010630641
> MSDA
>
> Competencies
> 4033.2.1 : Storytelling with Data
>   The graduate communicates data insights to technical and nontechnical audiences.
>
> 4033.2.2 : Data Visualizations and Representations
>   The graduate creates data representations to offer insight into an organizational problem.
>
> 4033.2.3 : Dashboards
>   The graduate designs interactive dashboards to support executive decision-making.

#### Table of Contents
<ul>
    <li><a href="#interactice-data-dashboard">A1: Interactive Data Dashboard</li>
    <li><a href="#data-sets">A2: Data Sets</a></li>
    <li><a href="#installation-instructions">A3: Installation Instructions</a></li>
    <li><a href="#panopto-storying-telling-with-data">B: Panopto Storying Telling With Data</a></li>
    <li><a href="#dashboard-alignment">C1: Dashboard Alignment</a></li>
    <li><a href="#additional-data-set-insights">C2: Additional Data Set Insights</a></li>
    <li><a href="#decision-making-support">C3: Decision-Making Support</a></li>
    <li><a href="#interactice-controls">C4: Interactice Controls</a></li>
    <li><a href="#colorblindness">C5: Colorblindness</a></li>
    <li><a href="#data-representation">C6: Data Representation</a></li>
    <li><a href="#audience-analysis">C7: Audience Analysis</a></li>
    <li><a href="#universal-access">C8: Universal Access</a></li>
    <li><a href="#effective-storytelling">C9: Effective Storytelling</a></li>
    <li><a href="#sources">D: Sources</a></li>
    <li><a href="#professional-communication">E: Professional Communication</a></li>
</ul>

<a id="interactice-data-dashboard"></a>
# A1: Interactive Data Dashboard

<a id="data-sets"></a>
# A2: Data Sets

<a id="installation-instructions"></a>
# A3: Installation Instructions

<a id="panopto-storying-telling-with-data"></a>
# B: Panopto Storying Telling With Data

<a id="dashboard-alignment"></a>
# C1: Dashboard Alignment

<a id="additional-data-set-insights"></a>
# C2: Additional Data Set Insights

<a id="decision-making-support"></a>
# C3: Decision-Making Support

<a id="interactice-controls"></a>
# C4: Interactive Controls

<a id="colorblindness"></a>
# C5: Colorblindness

<a id="data-representation"></a>
# C6: Data Representation

<a id="audience-analysis"></a>
# C7: Audience Analysis

<a id="universal-access"></a>
# C8: Universal Access

<a id="effective-storytelling"></a>
# C9: Effective Storytelling

<a id="sources"></a>
# D: Sources

<a id="professional-communication"></a>
# D: Professional Communication